In [8]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.presidency.ucsb.edu/statistics/data/presidential-job-approval-all-data'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Use a set to ensure uniqueness of links
approval_links = set()

# Find all links and filter for those ending with 'public-approval'
for link in soup.find_all('a', href=True):
    if link['href'].endswith('public-approval'):
        approval_links.add(link['href'])  # Uniqueness based on link

# Print the unique filtered links
for link in approval_links:
    print(f'{link}')

https://www.presidency.ucsb.edu/statistics/data/joseph-r-biden-public-approval
https://www.presidency.ucsb.edu/statistics/data/george-bush-public-approval
https://www.presidency.ucsb.edu/statistics/data/dwight-d-eisenhower-public-approval
https://www.presidency.ucsb.edu/statistics/data/richard-m-nixon-public-approval
https://www.presidency.ucsb.edu/statistics/data/barack-obama-public-approval
https://www.presidency.ucsb.edu/statistics/data/lyndon-b-johnson-public-approval
https://www.presidency.ucsb.edu/statistics/data/john-f-kennedy-public-approval
https://www.presidency.ucsb.edu/statistics/data/jimmy-carter-public-approval
https://www.presidency.ucsb.edu/statistics/data/gerald-r-ford-public-approval
https://www.presidency.ucsb.edu/statistics/data/ronald-reagan-public-approval
https://www.presidency.ucsb.edu/statistics/data/harry-s-truman-public-approval
https://www.presidency.ucsb.edu/statistics/data/william-j-clinton-public-approval
https://www.presidency.ucsb.edu/statistics/data/do

In [9]:
import pandas as pd

def get_approval_data(link):
    # Request the page
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table in the page
    table = soup.find('table')

    # Parse the table into a DataFrame
    rows = []
    headers = [header.text.strip() for header in table.find_all('th')]
    
    for row in table.find_all('tr')[1:]:
        cols = [col.text.strip() for col in row.find_all('td')]
        rows.append(cols)
    
    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=headers)
    return df

# Example: Fetching Joseph R. Biden's approval data
link = 'https://www.presidency.ucsb.edu/statistics/data/joseph-r-biden-public-approval'
df_biden = get_approval_data(link)

# Display the DataFrame
print(df_biden)

    Start Date    End Date Approving Disapproving Unsure/NoData
0     9/3/2024   9/15/2024        39           58             3
1     8/1/2024   8/20/2024        43           53             4
2     7/1/2024   7/21/2024        36           58             6
3     6/3/2024   6/23/2024        38           58             4
4     5/1/2024   5/23/2024        39           56             5
5     4/1/2024   4/22/2024        38           58             4
6     3/1/2024   3/20/2024        40           55             5
7     2/1/2024   2/20/2024        38           59             3
8     1/2/2024   1/22/2024        41           54             5
9    12/1/2023  12/20/2023        39           59             2
10   11/1/2023  11/20/2023        37           59             4
11   10/2/2023  10/23/2023        37           59             4
12    9/1/2023   9/23/2023        41           58             1
13    8/1/2023   8/23/2023        42           53             5
14    7/3/2023   7/27/2023        40    

In [10]:
# Iterate over each link, scrape the data, and combine into one DataFrame
all_data = pd.DataFrame()
for link in approval_links:
    df = get_approval_data(link)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Display the combined DataFrame
print(all_data)

     Start Date   End Date Approving Disapproving Unsure/NoData     
0      9/3/2024  9/15/2024        39           58             3  NaN
1      8/1/2024  8/20/2024        43           53             4  NaN
2      7/1/2024  7/21/2024        36           58             6  NaN
3      6/3/2024  6/23/2024        38           58             4  NaN
4      5/1/2024  5/23/2024        39           56             5  NaN
...         ...        ...       ...          ...           ...  ...
1899   3/9/2001  3/11/2001        58           29            13  NaN
1900   3/5/2001   3/7/2001        63           22            15  NaN
1901  2/19/2001  2/21/2001        62           21            17  NaN
1902   2/9/2001  2/11/2001        57           25            18  NaN
1903   2/1/2001   2/4/2001        57           25            18  NaN

[1904 rows x 6 columns]


In [11]:
# Convert 'Start Date' and 'End Date' to datetime format for easier filtering
all_data['Start Date'] = pd.to_datetime(all_data['Start Date'], errors='coerce')
all_data['End Date'] = pd.to_datetime(all_data['End Date'], errors='coerce')

# Filter rows where either Start Date or End Date is in September
september_data = all_data[
    (all_data['Start Date'].dt.month == 9) | (all_data['End Date'].dt.month == 9)
]

# Display the filtered DataFrame
print(september_data)

     Start Date   End Date Approving Disapproving Unsure/NoData     
0    2024-09-03 2024-09-15        39           58             3  NaN
12   2023-09-01 2023-09-23        41           58             1  NaN
23   2022-09-01 2022-09-16        42           56             2  NaN
35          NaT 2021-09-17        43           55             2  NaN
50   1992-09-17 1992-09-20        36           54            10  NaN
...         ...        ...       ...          ...           ...  ...
1842 2002-09-05 2002-09-08        66           30             4  NaN
1843 2002-09-02 2002-09-04        66           29             5  NaN
1881 2001-09-21 2001-09-22        90            6             4  NaN
1882 2001-09-14 2001-09-15        86           10             4  NaN
1883 2001-09-07 2001-09-10        51           39            10  NaN

[172 rows x 6 columns]


In [13]:
# Assuming 'september_data' already contains only rows from September

# Extract the year from the 'Start Date' in september_data
september_data.loc[:, 'Year'] = september_data['Start Date'].dt.year

# Define the list of years you're interested in
years_of_interest = [1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 
                     1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]

# Filter the September data for only the years of interest
filtered_september_data = september_data[september_data['Year'].isin(years_of_interest)]

# Display the filtered DataFrame
print(filtered_september_data)

     Start Date   End Date Approving Disapproving Unsure/NoData         Year
50   1992-09-17 1992-09-20        36           54            10  NaN  1992.0
51   1992-09-11 1992-09-15        39           55             6  NaN  1992.0
52   1992-08-31 1992-09-02        39           54             7  NaN  1992.0
387  2016-09-26 2016-10-02        53           44             3  NaN  2016.0
388  2016-09-19 2016-09-25        53           43             4  NaN  2016.0
389  2016-09-12 2016-09-18        52           44             4  NaN  2016.0
390  2016-09-05 2016-09-11        51           45             4  NaN  2016.0
391  2016-08-29 2016-09-04        52           44             4  NaN  2016.0
596  2012-09-24 2012-09-30        48           45             7  NaN  2012.0
597  2012-09-17 2012-09-23        49           45             6  NaN  2012.0
598  2012-09-10 2012-09-16        50           44             6  NaN  2012.0
599  2012-09-03 2012-09-09        50           44             6  NaN  2012.0

In [14]:
# Ensure 'End Date' is in datetime format for proper sorting
filtered_september_data.loc[:, 'End Date'] = pd.to_datetime(filtered_september_data['End Date'], errors='coerce')

# Sort the data by 'Year' and 'End Date' to ensure the latest entry comes last
filtered_september_data = filtered_september_data.sort_values(by=['Year', 'End Date'], ascending=[True, False])

# Drop duplicates to keep only the latest entry for each year
latest_september_data = filtered_september_data.drop_duplicates(subset='Year', keep='first')

# Display the latest data for each year
print(latest_september_data)

     Start Date   End Date Approving Disapproving Unsure/NoData         Year
793  1968-09-19 1968-09-24        42           51             7  NaN  1968.0
915  1980-09-12 1980-09-15        37           55             8  NaN  1980.0
1098 1984-09-28 1984-10-01        54           35            11  NaN  1984.0
1043 1988-09-25 1988-10-01        54           37             9  NaN  1988.0
50   1992-09-17 1992-09-20        36           54            10  NaN  1992.0
1362 1996-09-07 1996-09-09        60           31             9  NaN  1996.0
1250 2000-09-11 2000-09-13        60           34             6  NaN  2000.0
1762 2004-09-24 2004-09-26        54           44             2  NaN  2004.0
1631 2008-09-26 2008-09-27        27           69             4  NaN  2008.0
596  2012-09-24 2012-09-30        48           45             7  NaN  2012.0
387  2016-09-26 2016-10-02        53           44             3  NaN  2016.0
1463 2020-09-30 2020-10-15        43           55             2  NaN  2020.0

In [16]:
latest_september_data.shape

(12, 7)

In [20]:
# Create a new DataFrame with just 'Year' and 'Approving' (renaming it to 'sepapp')
final_df = latest_september_data[['Year', 'Approving']].rename(columns={'Approving': 'sepapp', 'Year':'year'})

# Display the final DataFrame
print(final_df)

        year sepapp
793   1968.0     42
915   1980.0     37
1098  1984.0     54
1043  1988.0     54
50    1992.0     36
1362  1996.0     60
1250  2000.0     60
1762  2004.0     54
1631  2008.0     27
596   2012.0     48
387   2016.0     53
1463  2020.0     43


In [21]:
# Save the final DataFrame to a CSV file
final_df.to_csv('data/latest_september_data.csv', index=False)